In [1]:
import gzip
import os
import datetime
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook

/home/achang/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = None

def reset_tf(sess = None, log_device_placement = False):
    if sess:
        sess.close()
    tf.reset_default_graph()
    tf.set_random_seed(0)
    return tf.InteractiveSession(config = tf.ConfigProto(log_device_placement = log_device_placement))

def dump_statistics():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        print('parameters for "%s": %d' % (variable.name, variable_parameters))
        total_parameters += variable_parameters
    print('total parameters: %d' % total_parameters)

In [3]:
class HyperParameters:
    learning_rate = 1e-3
    
    dropout_rate = 0.1
    
    context_size = 850
    question_size = 60
    answers_size = 6
    
    d_hidden = 100
    
    num_rnn_layers_contexts = 1
    num_rnn_layers_questions = 1

    dataset_batch_size = 64
    dataset_num_parallel_calls = 4
    dataset_prefetch_size = 1000
    dataset_shuffle_size = 1000
    
    max_distance_bias = 15
    
    gradient_clip_norm = 5.0

In [ ]:
class AttentionModel:
    def __init__(self, session, word_embeddings, hparams):
        self._session = session
        self._word_embeddings = word_embeddings
        self._hparams = hparams
        
    def _parse_example(self, example_proto):
        # parse proto
        parsed = tf.parse_single_example(example_proto, features = {
            'context': tf.VarLenFeature(tf.int64),
            'question': tf.VarLenFeature(tf.int64),
            'answer_starts': tf.VarLenFeature(tf.int64),
            'answer_ends': tf.VarLenFeature(tf.int64), })
        
        # convert to dense tensors
        context = tf.sparse_tensor_to_dense(parsed['context'])
        question = tf.sparse_tensor_to_dense(parsed['question'])
        answer_starts = tf.sparse_tensor_to_dense(parsed['answer_starts'])
        answer_ends = tf.sparse_tensor_to_dense(parsed['answer_ends'])
        
        # pad tensors
        context_len = tf.shape(context)[0]
        question_len = tf.shape(question)[0]
        answers_len = tf.shape(answer_starts)[0]
        zero_vector = self._word_embeddings.shape[0] - 1
        context = tf.pad(
            context,
            [[0, self._hparams.context_size - context_len]],
            constant_values = 0)
        question = tf.pad(
            question,
            [[0, self._hparams.question_size - question_len]],
            constant_values = 0)
        answer_starts = tf.pad(
            answer_starts,
            [[0, self._hparams.answers_size - answers_len]],
            constant_values = -1)
        answer_ends = tf.pad(
            answer_ends,
            [[0, self._hparams.answers_size - answers_len]],
            constant_values = -1)
        
        return (context, context_len, question, question_len, answer_starts, answer_ends)
    
    def _build_dataset_pipeline(self):
        with tf.variable_scope('dataset'):
            # placeholders
            self._dataset_filenames = tf.placeholder(
                tf.string,
                shape = [None],
                name = 'dataset_filenames')
            self._dataset_limit = tf.placeholder_with_default(
                tf.constant(-1, tf.int64),
                shape = [],
                name = 'dataset_limit')
            self._dataset_shuffle_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_batch_size, tf.int64),
                shape = [],
                name = 'dataset_shuffle_size')
            self._dataset_batch_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_batch_size, tf.int64),
                shape = [],
                name = 'dataset_batch_size')
            self._dataset_prefetch_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_prefetch_size, tf.int64),
                shape = [],
                name = 'dataset_prefetch_size')

            # build dataset
            dataset = tf.data.TFRecordDataset(
                tf.random_shuffle(self._dataset_filenames),
                compression_type='GZIP')
            dataset = dataset.take(self._dataset_limit)
            dataset = dataset.map(
                self._parse_example,
                num_parallel_calls = self._hparams.dataset_num_parallel_calls)
            dataset = dataset.shuffle(self._dataset_shuffle_size)
            dataset = dataset.prefetch(self._dataset_prefetch_size)
            dataset = dataset.batch(self._dataset_batch_size)

            # build iterator
            self._dataset_iterator = dataset.make_initializable_iterator()
            (contexts,
             context_lens,
             questions,
             question_lens,
             answer_starts,
             answer_ends) = self._dataset_iterator.get_next()
            
            # give key tensors names
            self._contexts = tf.identity(contexts, 'contexts')
            self._context_lens = tf.identity(context_lens, 'context_lens')
            self._questions = tf.identity(questions, 'questions')
            self._question_lens = tf.identity(question_lens, 'question_lens')
            self._answer_starts = tf.identity(answer_starts, 'answer_starts')
            self._answer_ends = tf.identity(answer_ends, 'answer_ends')

            # hint static shapes
            self._contexts.set_shape([None, self._hparams.context_size])
            self._questions.set_shape([None, self._hparams.question_size])
            self._answer_starts.set_shape([None, self._hparams.answers_size])
            self._answer_ends.set_shape([None, self._hparams.answers_size])

            # minibatch size
            self._minibatch_size = tf.shape(self._contexts)[0]
            self._minibatch_size = tf.identity(self._minibatch_size, 'minibatch_size')
    
    def _bidirectional_rnn_layer(self, layer, lens, size):
        # TODO: layer norm
        
        forward = tf.nn.rnn_cell.GRUCell(size)
        forward = tf.nn.rnn_cell.DropoutWrapper(
            forward,
            input_keep_prob = self._dropout_keep_prob,
            output_keep_prob = self._dropout_keep_prob)
        backward = tf.nn.rnn_cell.GRUCell(size)
        backward = tf.nn.rnn_cell.DropoutWrapper(
            backward,
            input_keep_prob = self._dropout_keep_prob,
            output_keep_prob = self._dropout_keep_prob)

        # bidirectional RNN
        layer, _ = tf.nn.bidirectional_dynamic_rnn(
            forward,
            backward,
            layer,
            sequence_length = lens,
            dtype = tf.float32)
        layer = tf.concat(layer, 2)
        
        return layer

    def _build_model(self):
        with tf.variable_scope('model'):
            # placeholders
            self._training = tf.placeholder(tf.bool, name = 'training')
            
            # dropout keep probability
            self._dropout_keep_prob = tf.cond(
                self._training,
                lambda: tf.constant(1.0),
                lambda: tf.constant(1.0 - self._hp.dropout_rate))
            self._dropout_keep_prob = tf.identity(
                self._dropout_keep_prob,
                'dropout_keep_prob')
            
            # init embedding
            word_embeddings = tf.get_variable(
                name = "word_embeddings",
                shape = self._word_embeddings.shape,
                initializer = tf.constant_initializer(self._word_embeddings),
                trainable = False)
            
            # embed contexts/questions
            contexts_embedded = tf.nn.embedding_lookup(
                word_embeddings,
                self._contexts)
            questions_embedded = tf.nn.embedding_lookup(
                word_embeddings,
                self._questions)
            
            # context RNN layers
            contexts_layer = contexts_embedded
            for i in range(self._hparams.num_rnn_layers_contexts):
                with tf.variable_scope('contexts_rnn_%d' % i):
                    contexts_layer = self._bidirectional_rnn_layer(
                        contexts_layer,
                        self._context_lens,
                        self._hparams.d_hidden)

            # question RNN layers
            questions_layer = questions_embedded
            for i in range(self._hparams.num_rnn_layers_questions):
                with tf.variable_scope('questions_rnn_%d' % i):
                    questions_layer = self._bidirectional_rnn_layer(
                        questions_layer,
                        self._questions_lens,
                        self._hparams.d_hidden)
                    
            # joint attention layer
            with tf.variable_scope('joint'):
                joint_layer = self._attention_layer(
                    queries = contexts_layer,
                    keys = questions_layer,
                    values = questions_layer)
                joint_layer = self._fusion_layer(
                    contexts_layer,
                    joint_layer,
                    self._hparams.d_hidden)
                
            # joint self-attention layers
            for i in range(self._hparams.num_attn_layers_joint):
                with tf.variable_scope('joint_self_%d' % i):
                    joint_layer = self._attention_layer_self(joint_layer)

            # output: start/end logits
            self._answer_start_logits = tf.layers.dense(
                joint_layer,
                1,
                use_bias = False,
                name = 'answer_start_logits')
            self._answer_start_logits = tf.squeeze(    # [batch_size, context_size]
                self._answer_start_logits,
                axis = -1,
                name = 'answer_start_logits') 
            self._answer_end_logits = tf.layers.dense(
                joint_layer,
                1,
                use_bias = False,
                name = 'answer_end_logits')
            self._answer_end_logits = tf.squeeze(      # [batch_size, context_size]
                self._answer_end_logits,
                axis = -1,
                name = 'answer_end_logits')

    def _build_optimizer(self):
        with tf.variable_scope('optimize'):
            # individual losses
            # TODO: incorporate other answers into training
            l0 = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = self._answer_starts[:, 0],
                logits = self._answer_start_logits)
            l1 = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = self._answer_ends[:, 0],
                logits = self._answer_end_logits)

            # total loss
            self._total_loss = tf.reduce_sum(l0) + tf.reduce_sum(l1)
            self._total_loss = tf.identity(self._total_loss, 'total_loss')
            
            # mean loss
            self._mean_loss = self._total_loss / tf.cast(self._minibatch_size, tf.float32)
            self._mean_loss = tf.identity(self._mean_loss, 'mean_loss')
            
            # start/end probabilities/estimates
            self._answer_start_probs = tf.nn.softmax(
                self._answer_start_logits,
                name = 'answer_start_logits')
            self._answer_start_estimates = tf.argmax(
                self._answer_start_probs,
                axis = -1,
                name = 'answer_start_estimates')
            self._answer_end_probs = tf.nn.softmax(
                self._answer_end_logits,
                name = 'answer_end_logits')
            # N.B., mask impossible answers
            mask = 1.0 - tf.sequence_mask(
                self._answer_start_estimates,
                self._hparams.context_size,
                dtype = tf.float32)
            self._answer_end_estimates = tf.argmax(
                mask * self._answer_end_probs,
                axis = -1,
                name = 'answer_end_estimates')
            
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self._global_step = tf.Variable(0, name = 'global_step', trainable = False)
                self._optimizer = tf.train.AdamOptimizer(learning_rate = self._hparams.learning_rate)
                
                # gradient clipping
                gradients, variables = zip(*self._optimizer.compute_gradients(self._mean_loss))
                gradients, _ = tf.clip_by_global_norm(
                    gradients, 
                    self._hparams.gradient_clip_norm)
                
                self._train_op = self._optimizer.apply_gradients(
                    zip(gradients, variables),
                    global_step = self._global_step)
                
    def _compute_errors(self,
                        answer_start,
                        answer_end,
                        answer_start_estimate,
                        answer_end_estimate):
        # shorter notation
        a0 = answer_start
        a1 = answer_end
        al = a1 - a0 + 1
        b0 = answer_start_estimate
        b1 = answer_end_estimate
        bl = b1 - b0 + 1
        
        assert a0 <= a1
        assert b0 <= b1
        
        # true positives: compute overlap
        tp = max(0, min(a1+1, b1+1) - max(a0, b0))
        # false positives: leftovers from b
        fp = bl - tp
        # false negatives: leftovers from a
        fn = al - tp
                
        return (tp, fp, fn)

    def process(self,
                dataset_filenames,
                dataset_limit = -1,
                header = 'results',
                train = False,
                log_file = None):
        # initialize dataset to files
        self._session.run(self._dataset_iterator.initializer, feed_dict={
            self._dataset_filenames: dataset_filenames,
            self._dataset_limit: dataset_limit })

        cum_loss = 0
        cum_num_examples = 0
        cum_exact_matches = 0
        cum_tps = 0
        cum_fps = 0
        cum_fns = 0
        
        # start progress
        start = datetime.datetime.now()
        progress = tqdm_notebook(leave = False, desc = header)

        while True:
            # process a minibatch
            try:
                (_,
                 curr_total_loss, 
                 answer_starts,
                 answer_ends,
                 answer_start_estimates,
                 answer_end_estimates) = self._session.run(
                    (self._train_op if train else (),
                     self._total_loss,
                     self._answer_starts[:, 0],
                     self._answer_ends[:, 0],
                     self._answer_start_estimates,
                     self._answer_end_estimates),
                    feed_dict = { self._training: train })
            except tf.errors.OutOfRangeError:
                break
                
            print(answer_starts)
            print(answer_ends)

            # update loss stats
            cum_loss += curr_total_loss
            cum_num_examples += answer_starts.size
            
            # update exact match stats
            for i in range(answer_starts.size):
                if (answer_starts[i] == answer_start_estimates[i]
                    and answer_ends[i] == answer_end_estimates[i]):
                    cum_exact_matches += 1

            # update F1 stats
            for i in range(answer_starts.size):
                tps, fps, fns = self._compute_errors(
                    answer_starts[i],
                    answer_ends[i],
                    answer_start_estimates[i],
                    answer_end_estimates[i])
                cum_tps += tps
                cum_fps += fps
                cum_fns += fns

            # update progress
            progress.update(answer_starts.size)

        # end progress
        progress.close()
        finish = datetime.datetime.now()
        
        # precision
        precision = 0
        if cum_tps + cum_fps > 0:
            precision = cum_tps / (cum_tps + cum_fps)
            
        # recall
        recall = 0
        if cum_tps + cum_fns > 0:
            recall = cum_tps / (cum_tps + cum_fns)
            
        # F1
        F1 = 0
        if precision + recall > 0:
            F1 = 2 * precision * recall / (precision + recall)
        
        # print/log output
        message = '%s: time=%s, step=%d, loss=%g, exact_match=%g, precision=%g, recall=%g, F1=%g' % (
            header,
            finish - start,
            tf.train.global_step(sess, self._global_step),
            cum_loss / cum_num_examples,
            cum_exact_matches / cum_num_examples,
            precision,
            recall,
            F1)
        print(message)
        if log_file:
            print(message, file=log_file)
            log_file.flush()

In [5]:
with gzip.open('../../data/SQuAD/data_1.vocab.embeddings.npy.gz', 'rb') as f:
    word_embeddings = np.load(f)

In [6]:
def list_files(path):
    return sorted([os.path.join(path, file) for file in os.listdir(path)])

train_set = list_files('../../data/SQuAD/data_1.train')
dev_set = list_files('../../data/SQuAD/data_1.dev')

In [ ]:
sess = reset_tf(sess)

model = AttentionModel(sess, word_embeddings, HyperParameters())
model._build_dataset_pipeline()
model._build_model()
model._build_optimizer()
dump_statistics()

In [11]:
sess.run(tf.global_variables_initializer())

In [10]:
for i in range(100):
    model.process(train_set[:1], dataset_limit = 10, train = True)

HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results: time=0:00:00.786739, step=11, loss=2.07409, exact_match=0.7, precision=0.345455, recall=0.76, F1=0.475


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results: time=0:00:00.765898, step=12, loss=2.49023, exact_match=0.5, precision=0.0683761, recall=0.32, F1=0.112676


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

...

results: time=0:00:00.822098, step=108, loss=0.826614, exact_match=0.8, precision=0.25, recall=0.72, F1=0.371134


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results: time=0:00:00.852039, step=109, loss=0.49378, exact_match=0.8, precision=0.6, recall=0.72, F1=0.654545


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results: time=0:00:00.836542, step=110, loss=0.787361, exact_match=0.8, precision=0.178218, recall=0.72, F1=0.285714


In [12]:
sess.run(
    model._dataset_iterator.initializer,
    feed_dict = {
        model._dataset_filenames: train_set[:1],
        model._dataset_limit: 10 })

In [14]:
contexts, context_lens, questions, question_lens, answer_starts, answer_ends, answer_start_estimates, answer_end_estimates = sess.run(
    [model._contexts,
     model._context_lens,
     model._questions,
     model._question_lens,
     model._answer_starts,
     model._answer_ends,
     model._answer_start_estimates,
     model._answer_end_estimates],
    feed_dict = { model._training: False })

In [16]:
contexts

array([[   5,  571,    2, ...,    0,    0,    0],
       [  36, 1448, 2230, ...,    0,    0,    0],
       [   5, 3769,   87, ...,    0,    0,    0],
       ...,
       [  69,   77,   37, ...,    0,    0,    0],
       [   1, 9191, 2659, ...,    0,    0,    0],
       [ 181,  832,  562, ...,    0,    0,    0]])

In [57]:
answer_end_estimates

array([ 29,   5,  78,  43,  34,  25, 124, 117,  55, 110])

In [58]:
answer_starts[:, 0]

array([ 65,   4,  78,  49,  80, 181, 123, 117,  52, 110])

In [59]:
answer_ends[:, 0]

array([ 67,   5,  78,  49,  80, 189, 124, 117,  55, 110])